In [ ]:
import xgboost as xgb
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import parallel_backend
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
import plotly as py
import csv

In [ ]:
# Load dataset
data = pd.read_csv("Mixed toxicity modeling.csv", sep=",", header=0)
y = data.iloc[:, -1]
X = data.iloc[:, 1:-1]

In [ ]:
# Split dataset into training and testing sets
bins = np.linspace(0, 1.4, 14)
y_binned = np.digitize(y, bins)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y_binned, random_state=13)

In [ ]:
# Define model
model = xgb.XGBRegressor()

In [ ]:
# Perform 10-fold cross-validation
scores = cross_val_score(model, X_train, y_train, cv=10, scoring='r2', error_score='raise')
print(f'Mean R^2 score: {np.mean(scores)}')
print(f'Standard deviation: {np.std(scores)}')

In [ ]:
# Grid search for hyperparameter tuning
param_grid = {
    'n_estimators': [700, 800, 900],
    'max_depth': [2, 3, 4],
    'learning_rate': [0.1, 0.2, 0.3]
}

In [ ]:
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='neg_mean_squared_error')
with parallel_backend('multiprocessing', n_jobs=-1):
    grid_search.fit(X_train, y_train)

In [ ]:
# Output best parameters and score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", -grid_search.best_score_)

In [ ]:
# Train model using best parameters
best_model = xgb.XGBRegressor(**grid_search.best_params_)
best_model.fit(X_train, y_train)

In [ ]:
# Evaluate model performance on training set
scores = cross_val_score(best_model, X_train, y_train, cv=10)
std_error = np.std(scores / np.sqrt(len(scores)))
print("Standard Error: ", std_error)
print("CV Score: ", np.mean(scores))

In [ ]:
# Predict on test set
y_pred = best_model.predict(X_test)

In [ ]:
# Calculate Mean Squared Error and RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

In [ ]:
# Calculate R^2 and adjusted R^2 scores
test_score = r2_score(y_test, y_pred)
n = len(y_test)
k = X_test.shape[1]
adjusted_r2 = 1 - ((1 - test_score) * (n - 1) / (n - k - 1))
print(f'Test Set R^2 Score: {test_score}')
print(f'Adjusted R^2 Score: {adjusted_r2}')
print("Mean Squared Error: ", mse)
print("Root Mean Squared Error: ", rmse)

In [ ]:
# Save output to CSV
output_matrix = np.concatenate((y_test.values.reshape(-1, 1), y_pred.reshape(-1, 1)), axis=1)
output_df = pd.DataFrame(output_matrix, columns=['y_test', 'y_pred'])
output_df.to_csv("XGBoost_output.csv")

In [ ]:
# Initialize SHAP and Plotly
shap.initjs()
py.offline.init_notebook_mode(connected=True)

In [ ]:
# Prepare data for SHAP
X_display = pd.DataFrame(X_test.values, columns=X_test.columns, index=X_test.index)
background_data = shap.maskers.Independent(X_train)
explainer = shap.Explainer(best_model, background_data)
shap_values = explainer(X_test)

In [ ]:
# Generate SHAP summary plot
plt.figure(dpi=300, figsize=(20, 10))
shap.summary_plot(shap_values, X_display, show=False, max_display=20)
plt.yticks(fontsize=8)
plt.savefig("XGBoost_shap_summary_plot.pdf", format="pdf")
plt.show()

In [ ]:
# Generate SHAP bar plot
plt.figure(dpi=500, figsize=(20, 10))
shap.plots.bar(shap_values, max_display=21, show=False)
plt.yticks(fontsize=8)
plt.savefig("XGBoost_shap_bar_plot.pdf", format="pdf")
plt.show()

In [ ]:
# Compute average SHAP values
shap_values_matrix = shap_values.values
feature_names = X_test.columns.tolist()
shap_means = np.mean(np.abs(shap_values_matrix), axis=0)
shap_dict = dict(zip(feature_names, shap_means))
with open('XGBoost_shap_values_mean.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['feature', 'shap_mean'])
    for key, value in shap_dict.items():
        writer.writerow([key, value])

In [ ]:
###Compute distance matrix and standardized residuals

In [ ]:
# Normalize data
def normalize_data(A):
    A_min = A.min(axis=0)
    A_max = A.max(axis=0)
    return (A - A_min) / (A_max - A_min)

In [ ]:
# Z-score normalization
def z_score_normalize(data):
    means = np.mean(data, axis=0)
    std_devs = np.std(data, axis=0)
    return (data - means) / std_devs

In [ ]:
# Compute Euclidean distances
def euclidean_distances(A):
    return np.sqrt(np.sum((A[:, np.newaxis, :] - A[np.newaxis, :, :]) ** 2, axis=2))

def average_distances_to_all_points(A):
    dist_matrix = euclidean_distances(A)
    sum_distances = np.sum(dist_matrix, axis=1)
    average_distances = sum_distances / (A.shape[0] - 1)
    return average_distances

In [ ]:
# Compute standardized residuals for training set
y_train_pred = best_model.predict(X_train)
residuals = y_train - y_train_pred
standardized_residuals_z = z_score_normalize(residuals)
np.savetxt("XGBoost_train_standardized_residuals.csv", standardized_residuals_z, delimiter=",")

In [ ]:
# Compute training set distance matrix
A = X_train.values
A_normalized = z_score_normalize(A)
avg_dists_to_all_normalized = average_distances_to_all_points(A_normalized)
np.savetxt("XGB_Train_avg_distances_to_all.csv", avg_dists_to_all_normalized, delimiter=",")

In [1]:
# Compute average distances from test set to training set
def euclidean_distances(A, B):
    A_np = A.to_numpy() if isinstance(A, pd.DataFrame) else A
    B_np = B.to_numpy() if isinstance(B, pd.DataFrame) else B
    return np.sqrt(np.sum((A_np[:, np.newaxis, :] - B_np[np.newaxis, :, :]) ** 2, axis=2))

def average_distances_from_B_to_A(A, B):
    dist_matrix = euclidean_distances(A, B)
    sum_distances = np.sum(dist_matrix, axis=0)
    average_distances = sum_distances / A.shape[0]
    return average_distances

B_normalized = z_score_normalize(X_test)
average_distances_from_B_to_A = average_distances_from_B_to_A(X_train, B_normalized)
np.savetxt("XGBoost_test_average_distances_from_B_to_A.csv", average_distances_from_B_to_A, delimiter=",")

In [ ]:
# Compute standardized residuals for test set
test_residuals = y_pred - y_test
test_standardized_residuals_z = z_score_normalize(test_residuals)
np.savetxt("XGBoost_test_standardized_residuals.csv", test_standardized_residuals_z, delimiter=",")